**`TEAM MEMBERS: ASAD AALI, ANKIT GUPTA, ANURAG GUPTA, STUTI KHANNA, YAMINI JHA, CHETNA YADAV`**

# Task A

Scrape Instagram.py to fetch ~200 posts from the Zara Instagram page. This script fetches (i) image URLs, (ii) post caption (the text description of a post), and (iii) # likes. 

In [ ]:
!pip install instaloader

     |████████████████████████████████| 54 kB 2.2 MB/s 
  Created wheel for instaloader: filename=instaloader-4.8.1-py3-none-any.whl size=58722 sha256=608aff65c4b0c13b8647775455fe9f6d7016fb4086600dea2e573b976cec237f
  Stored in directory: /root/.cache/pip/wheels/67/cc/31/c4f2a966180c23b0f66985bb9a90c389d447bc1a5ddffd35c9
Successfully built instaloader


In [ ]:
import instaloader
import pandas as pd
from itertools import dropwhile, takewhile

L = instaloader.Instaloader()
L.login()
df = pd.DataFrame()

i = 0

for post in instaloader.Profile.from_username(L.context, 'zara').get_posts():
  df = df.append({'Caption': post.caption, 'Likes': post.likes, 'URL': post.url}, ignore_index=True)
  i = i + 1
  if i > 200:
    break

^Removed login credentials from above code after extracting data.

In [ ]:
from google.colab import files

df.to_excel('zara.xlsx')
files.download('zara.xlsx')

## Depdendecies

In [5]:
!pip install google-cloud-vision

     |████████████████████████████████| 370 kB 5.0 MB/s 
     |████████████████████████████████| 95 kB 3.4 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.5 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 1.5.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.2.2 which is incompatible.
google-cloud-language 1.2.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.

# Task B

Using the image URLs, obtain image labels (text) from Google Vision (cloud service)

In [1]:
import xlrd
from google.cloud import vision
from google.cloud.vision_v1 import types
import os
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Application_Credentials = "/content/drive/MyDrive/Colab Notebooks/Unstructured Data Analytics/Assignment 3/uda-assignment-3-project-8250e2fe8d10.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials
client = vision.ImageAnnotatorClient()
image = vision_v1.types.Image()

loc = ("/content/drive/MyDrive/Colab Notebooks/Unstructured Data Analytics/Assignment 3/zara.xlsx")
df = pd.read_excel(loc, index_col=0)
df['Labels'] = ''
index = 0

# Loop through every url, retrieve images and send to google vision

for url in df['URL']:
  image.source.image_uri = url
  response = client.label_detection(image=image)
  labels = response.label_annotations
  str_labels = ''

  for label in labels:
    str_labels = str_labels + label.description + ', '
  
  df['Labels'][index] = str_labels
  index = index + 1

# Task C

Create a column called binary (lowercase only) where value =1 (stands for high engagement) or 0 (stands for low engagement) based on whether the number of likes is above or below the median value.  

In [ ]:
df['binary'] = df['Likes'].apply(lambda x: 1 if x >= df['Likes'].median() else 0)

In [ ]:
from google.colab import files

df.to_excel('task3-zara.xlsx')
files.download('task3-zara.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task D

Run a logistic regression with binary as the dependent variable, and the image_labels as independent variables. Before running the regression, replace the column label image_labels with text, since the script expects text to be the name for the column containing text. What is the accuracy (show the confusion matrix) of this prediction model? The idea is to be able to predict the engagement level for an image. 

Accuracy = 1 - # prediction errors / total # cases

What accuracy do you get by using the post_caption words as the independent variables instead of image_labels? As in the first regression, change post_caption to text before running the logistic regression. Finally, what accuracy do you get by combining (concatenating) the image_labels and post_caption and using them together as independent variables? What can you conclude from your analysis?

In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Unstructured Data Analytics/task3-zara.xlsx', index_col=0)
df

,Caption,Likes,URL,Labels,binary
0,Zara SRPLS Fall ‘21. Drop 02 now available #za...,7599,https://scontent-amt2-1.cdninstagram.com/v/t51...,"Forehead, Nose, Cheek, Chin, Eyebrow, Eyelash,...",0
1,Zara SRPLS Fall ‘21. Drop 02 now available #za...,10139,https://scontent-amt2-1.cdninstagram.com/v/t51...,"Hair, Forehead, Ear, Gesture, Black hair, Corn...",0
2,Zara SRPLS Fall ‘21. Now available #zarasrpls,17172,https://scontent-amt2-1.cdninstagram.com/v/t51...,"Clothing, Hair, Face, Skin, Head, Lip, Chin, O...",0
3,Zara SRPLS Fall ‘21. Now available #zarasrpls,26557,https://scontent-amt2-1.cdninstagram.com/v/t51...,"Leg, Sleeve, Eyewear, Standing, Gesture, Stree...",0
4,Zara SRPLS Fall ‘21. Now available #zarasrpls,21524,https://scontent-amt2-1.cdninstagram.com/v/t51...,"Head, Outerwear, Shoe, Shoulder, Leg, Luggage ...",0
...,...,...,...,...,...
196,Crochet special edition #zarawoman,44975,https://scontent-amt2-1.cdninstagram.com/v/t51...,"Sky, Beige, Wood, Tints and shades, Facade, La...",1
197,THE BEAUTY COLLECTION. \nON THE LINE \nCreativ...,27591,https://scontent-amt2-1.cdninstagram.com/v/t51...,"Nose, Cheek, Skin, Lip, Lipstick, Eyelash, Mou...",0
198,THE BEAUTY COLLECTION. \nNow available \nStill...,22501,https://scontent-ams4-1.cdninstagram.com/v/t51...,"Liquid, Product, Cosmetics, Material property,...",0
199,THE BEAUTY COLLECTION. \nON THE LINE \nCreativ...,39416,https://scontent-amt2-1.cdninstagram.com/v/t51...,"Nose, Skin, Head, Lip, Lipstick, Chin, Shoulde...",0


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Calculating engagement with only image Labels

In [4]:
# Extracting Labels from output to calculate the engagement
new_df= df[['Labels']].copy()
# data preprocessing
new_df['Labelss'] = new_df['Labels'].str.replace(',','')
new_df

,Labels,Labelss
0,"Forehead, Nose, Cheek, Chin, Eyebrow, Eyelash,...",Forehead Nose Cheek Chin Eyebrow Eyelash Jaw N...
1,"Hair, Forehead, Ear, Gesture, Black hair, Corn...",Hair Forehead Ear Gesture Black hair Cornrows ...
2,"Clothing, Hair, Face, Skin, Head, Lip, Chin, O...",Clothing Hair Face Skin Head Lip Chin Outerwea...
3,"Leg, Sleeve, Eyewear, Standing, Gesture, Stree...",Leg Sleeve Eyewear Standing Gesture Street fas...
4,"Head, Outerwear, Shoe, Shoulder, Leg, Luggage ...",Head Outerwear Shoe Shoulder Leg Luggage and b...
...,...,...
196,"Sky, Beige, Wood, Tints and shades, Facade, La...",Sky Beige Wood Tints and shades Facade Landsca...
197,"Nose, Cheek, Skin, Lip, Lipstick, Eyelash, Mou...",Nose Cheek Skin Lip Lipstick Eyelash Mouth Fla...
198,"Liquid, Product, Cosmetics, Material property,...",Liquid Product Cosmetics Material property Fon...
199,"Nose, Skin, Head, Lip, Lipstick, Chin, Shoulde...",Nose Skin Head Lip Lipstick Chin Shoulder Eyeb...


In [5]:
# Concatenating all lables into a single array format to feed into the tfidf vectorization method
vectorizer_input = []
for i in range(201):
  strind_dt = new_df.iloc[i,1]
  vectorizer_input.append(strind_dt)
  
vectorizer_input

['Forehead Nose Cheek Chin Eyebrow Eyelash Jaw Neck Sleeve Ear ',
 'Hair Forehead Ear Gesture Black hair Cornrows Happy Child Personal protective equipment Caesar cut ',
 'Clothing Hair Face Skin Head Lip Chin Outerwear Hairstyle Smile ',
 'Leg Sleeve Eyewear Standing Gesture Street fashion Waist Collar Knee Blazer ',
 'Head Outerwear Shoe Shoulder Leg Luggage and bags Knee Bag Human body Neck ',
 'Outerwear Fur clothing Neck Sleeve Parka Street fashion Wig Fashion design Toy Liver ',
 'Neck Sleeve Grey Waist Font Denim Pattern Electric blue Chest Woolen ',
 'Building Black-and-white World Font Tower block Facade Urban design Condominium City Roof ',
 'Outerwear Arm Neck Sleeve Gesture Collar Jacket Overcoat Street fashion Fashion design ',
 'Face Smile Head Outerwear Arm Eye Product Organ Human body Sleeve ',
 'Toy Sleeve Waist Collar Bag Fashion design Luggage and bags Blazer Street fashion Doll ',
 'Lip Outerwear Eyebrow Cap White Black Flash photography Sleeve Headgear Style ',
 'F

In [6]:
# optional code for TFIDF with min_df 
# when we took the top 30 attributes from tfidf using the parameter min_df =0.1, the accuracy of the regression went upto 100%, as the features were reduced significantly from 390 to 30

tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words= 'english',min_df = 0.1)
tfidf_wm = tfidfvectorizer.fit_transform(vectorizer_input)
tfidf_tokens = tfidfvectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = range(201),columns = tfidf_tokens)
df_tfidfvect

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,black,body,design,dress,eyebrow,eyelash,face,fashion,flash,font,forehead,gesture,hair,happy,human,knee,leg,lip,neck,nose,outerwear,photography,shoulder,sky,sleeve,street,thigh,waist,white
0,0.000000,0.00000,0.000000,0.0,0.402916,0.420333,0.000000,0.000000,0.000000,0.0,0.451037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.394921,0.451037,0.000000,0.000000,0.000000,0.0,0.313285,0.00000,0.0,0.000000,0.0
1,0.364981,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.382947,0.335302,0.684181,0.373643,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0
2,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.557742,0.000000,0.000000,0.0,0.000000,0.000000,0.478984,0.000000,0.000000,0.000000,0.000000,0.451886,0.000000,0.000000,0.505270,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0
3,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.341611,0.000000,0.0,0.000000,0.383908,0.000000,0.000000,0.000000,0.444097,0.391680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.304549,0.39168,0.0,0.372985,0.0
4,0.000000,0.40555,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.351639,0.410764,0.362282,0.000000,0.355093,0.000000,0.382163,0.000000,0.373851,0.0,0.000000,0.00000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.00000,0.0,0.000000,0.0
197,0.000000,0.00000,0.000000,0.0,0.000000,0.499650,0.000000,0.000000,0.364589,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.451851,0.000000,0.536148,0.000000,0.354683,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0
198,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0
199,0.000000,0.00000,0.000000,0.0,0.487482,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.459903,0.000000,0.545703,0.000000,0.000000,0.503049,0.0,0.000000,0.00000,0.0,0.000000,0.0


In [7]:
# TFIDF with all features

tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words= 'english')
tfidf_wm = tfidfvectorizer.fit_transform(vectorizer_input)
tfidf_tokens = tfidfvectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = range(201),columns = tfidf_tokens)
df_tfidfvect

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abdomen,accessory,active,adaptation,aeolian,ancient,arm,arranging,art,artifact,artist,arts,artwork,asphalt,astronomical,atmospheric,automotive,aviation,azure,backlighting,bag,bags,balance,band,barefoot,basketball,beach,beard,beige,bicycle,bicycles,black,blazer,block,blue,boat,boating,boats,body,boot,...,toy,transport,travel,tree,triangle,trousers,truck,twig,umbrella,undershirt,underwater,urban,vegetable,vehicle,vertebrate,vest,violet,vision,visual,waist,waste,watch,water,watercraft,wear,wheel,whiskers,white,wig,wind,window,wings,wood,woodwind,woolen,workwear,world,wrap,yellow,zebra
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.172157,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.397283,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.257342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.383881,0.402112,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.249191,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.0,0.0,0.0,0.0,0.351008,0.0,0.0,0.0,0.213657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.351008,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.264349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.233999,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.207386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.000000

In [ ]:
y = df['binary']
y

0      0
1      0
2      0
3      0
4      0
      ..
196    1
197    0
198    0
199    0
200    1
Name: binary, Length: 201, dtype: int64

In [ ]:
# Logistic Regression 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(df_tfidfvect, y, test_size=0.33, random_state=42)
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
y_pred = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, y_test)
score

0.6268656716417911

Regression with only image lables gives an accuracy of 62%

In [ ]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[22, 13],
       [12, 20]])

# Engagement with Caption 

In [ ]:
# Extracting Caption from output to calculate the engagement
caption_df= df[['Caption']].copy()
caption_df['Captions'] = caption_df['Caption'].str.replace(',','')
caption_df

,Caption,Captions
0,Zara SRPLS Fall ‘21. Drop 02 now available #za...,Zara SRPLS Fall ‘21. Drop 02 now available #za...
1,Zara SRPLS Fall ‘21. Drop 02 now available #za...,Zara SRPLS Fall ‘21. Drop 02 now available #za...
2,Zara SRPLS Fall ‘21. Now available #zarasrpls,Zara SRPLS Fall ‘21. Now available #zarasrpls
3,Zara SRPLS Fall ‘21. Now available #zarasrpls,Zara SRPLS Fall ‘21. Now available #zarasrpls
4,Zara SRPLS Fall ‘21. Now available #zarasrpls,Zara SRPLS Fall ‘21. Now available #zarasrpls
...,...,...
196,Crochet special edition #zarawoman,Crochet special edition #zarawoman
197,THE BEAUTY COLLECTION. \nON THE LINE \nCreativ...,THE BEAUTY COLLECTION. \nON THE LINE \nCreativ...
198,THE BEAUTY COLLECTION. \nNow available \nStill...,THE BEAUTY COLLECTION. \nNow available \nStill...
199,THE BEAUTY COLLECTION. \nON THE LINE \nCreativ...,THE BEAUTY COLLECTION. \nON THE LINE \nCreativ...


In [ ]:
vectorizer_input_cap = []
for i in range(201):
  strind_dt_cap = caption_df.iloc[i,1]
  vectorizer_input_cap.append(strind_dt_cap)

vectorizer_input_cap

['Zara SRPLS Fall ‘21. Drop 02 now available #zarasrpls',
 'Zara SRPLS Fall ‘21. Drop 02 now available #zarasrpls',
 'Zara SRPLS Fall ‘21. Now available #zarasrpls',
 'Zara SRPLS Fall ‘21. Now available #zarasrpls',
 'Zara SRPLS Fall ‘21. Now available #zarasrpls',
 'ZARA SRPLS. Collection 07. DRP01 COMING SOON #zarasrpls @zarakids',
 'ZARA SRPLS. Collection 07. DRP01 COMING SOON #zarasrpls #zarawoman',
 'ZARA SRPLS. Collection 07. DRP01 COMING SOON #zarasrpls @zaraman',
 'ZARA KIDS+ LILMCKIMM. Now available\n“This collection is an ode to New York. Inspired by and dedicated to my kids and all the kids growing up in the downtown parks in New York city” - Alastair Mckimm. \n@amytroostphotography\n@alastairmckimm\n@zarakids\xa0#zarakids',
 'ZARA KIDS+ LILMCKIMM. Now available\n“This collection is an ode to New York. Inspired by and dedicated to my kids and all the kids growing up in the downtown parks in New York city” - Alastair Mckimm. \n@amytroostphotography\n@alastairmckimm\n@zarakids

In [ ]:
# TFIDF
# Removed stop words and took the top 28 features, by using min_df =0.1

tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words= 'english', min_df=0.1)
tfidf_wm = tfidfvectorizer.fit_transform(vectorizer_input_cap)
tfidf_tokens = tfidfvectorizer.get_feature_names()
df_tfidfvect_cap = pd.DataFrame(data = tfidf_wm.toarray(),index = range(201),columns = tfidf_tokens)
df_tfidfvect_cap

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,21,available,baron,baronandbaron,beauty,collection,creative,diane,direction,fabien,fabienbaron,fall,guidopalau,hair,karltempler,kendal,make,new,photography,studio,thank,thereisnobeautyonlybeauties,zara,zarabeauty,zarakids,zaraman,zarastudio,zarawoman
0,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
1,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
2,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
3,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
4,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0
197,0.000000,0.000000,0.0,0.256088,0.476513,0.131043,0.256088,0.259471,0.256088,0.0,0.0,0.000000,0.00000,0.225861,0.256088,0.259471,0.235625,0.0,0.262999,0.0,0.223590,0.252837,0.000000,0.262999,0.0,0.0,0.0,0.0
198,0.000000,0.412899,0.0,0.000000,0.412899,0.227098,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.455778,0.0,0.000000,0.438166,0.000000,0.455778,0.0,0.0,0.0,0.0
199,0.000000,0.000000,0.0,0.256088,0.476513,0.131043,0.256088,0.259471,0.256088,0.0,0.0,0.000000,0.00000,0.225861,0.256088,0.259471,0.235625,0.0,0.262999,0.0,0.223590,0.252837,0.000000,0.262999,0.0,0.0,0.0,0.0


In [ ]:
# Logistic Regression 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(df_tfidfvect_cap, y, test_size=0.33, random_state=42)
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
y_pred = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, y_test)
score

0.7014925373134329

The model with only captions gives an accuracy of 70%

In [ ]:
# Confusion Matrix

confusion_matrix(y_test, y_pred)

array([[26,  9],
       [11, 21]])

# Calculating engagement for Combined data (Labels +Caption)

In [ ]:
# Combining TFIDF data of Caption and Labels

df_combined = pd.concat([df_tfidfvect, df_tfidfvect_cap], axis=1)
df_combined

,abdomen,accessory,active,adaptation,aeolian,ancient,arm,arranging,art,artifact,artist,arts,artwork,asphalt,astronomical,atmospheric,automotive,aviation,azure,backlighting,bag,bags,balance,band,barefoot,basketball,beach,beard,beige,bicycle,bicycles,black,blazer,block,blue,boat,boating,boats,body,boot,...,wig,wind,window,wings,wood,woodwind,woolen,workwear,world,wrap,yellow,zebra,21,available,baron,baronandbaron,beauty,collection,creative,diane,direction,fabien,fabienbaron,fall,guidopalau,hair,karltempler,kendal,make,new,photography,studio,thank,thereisnobeautyonlybeauties,zara,zarabeauty,zarakids,zaraman,zarastudio,zarawoman
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.172157,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.397283,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.383881,0.402112,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.249191,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.561445,0.508624,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.553913,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.345342,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.0,0.0,0.0,0.0,0.351008,0.0,0.0,0.0,0.213657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.351008,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.233999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0
197,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.

In [ ]:
# Logistic Regression 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(df_combined, y, test_size=0.33, random_state=42)
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
y_pred = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, y_test)
score

0.7313432835820896

The model with image labels + captions gives an accuracy of 73%

In [ ]:
confusion_matrix(y_test, y_pred)

array([[27,  8],
       [10, 22]])

# Conclusion 
Accuracy with Labels : 0.62
Accuracy with Caption : 0.70
Accuracy with Labels + Caption : 0.73

Captions have higher engagement than labels with the likes. But both of them combined provide us with better prediction of number of likes.

In [8]:
selected_columns = df[["Labels","Likes"]]
df_capt_likes = selected_columns.copy()

df_capt_likes

,Labels,Likes
0,"Forehead, Nose, Cheek, Chin, Eyebrow, Eyelash,...",7599
1,"Hair, Forehead, Ear, Gesture, Black hair, Corn...",10139
2,"Clothing, Hair, Face, Skin, Head, Lip, Chin, O...",17172
3,"Leg, Sleeve, Eyewear, Standing, Gesture, Stree...",26557
4,"Head, Outerwear, Shoe, Shoulder, Leg, Luggage ...",21524
...,...,...
196,"Sky, Beige, Wood, Tints and shades, Facade, La...",44975
197,"Nose, Cheek, Skin, Lip, Lipstick, Eyelash, Mou...",27591
198,"Liquid, Product, Cosmetics, Material property,...",22501
199,"Nose, Skin, Head, Lip, Lipstick, Chin, Shoulde...",39416


In [9]:
# Load the regular expression library
import re
# Remove punctuation
df_capt_likes['Labels_processed'] = \
df_capt_likes['Labels'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_capt_likes['Labels_processed'] = \
df_capt_likes['Labels_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
df_capt_likes['Labels_processed']

0      forehead nose cheek chin eyebrow eyelash jaw n...
1      hair forehead ear gesture black hair cornrows ...
2      clothing hair face skin head lip chin outerwea...
3      leg sleeve eyewear standing gesture street fas...
4      head outerwear shoe shoulder leg luggage and b...
                             ...                        
196    sky beige wood tints and shades facade landsca...
197    nose cheek skin lip lipstick eyelash mouth fla...
198    liquid product cosmetics material property fon...
199    nose skin head lip lipstick chin shoulder eyeb...
200    eyebrow eyelash mouth jaw makeover iris black ...
Name: Labels_processed, Length: 201, dtype: object

In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
processed_labels = df_capt_likes['Labels_processed'].map(preprocess)
processed_labels

0      [forehead, nose, cheek, chin, eyebrow, eyelash...
1      [hair, forehead, gestur, black, hair, cornrow,...
2      [cloth, hair, face, skin, head, chin, outerwea...
3      [sleev, eyewear, stand, gestur, street, fashio...
4      [head, outerwear, shoe, shoulder, luggag, bag,...
                             ...                        
196    [beig, wood, tint, shade, facad, landscap, tre...
197    [nose, cheek, skin, lipstick, eyelash, mouth, ...
198    [liquid, product, cosmet, materi, properti, fo...
199    [nose, skin, head, lipstick, chin, shoulder, e...
200    [eyebrow, eyelash, mouth, makeov, iri, black, ...
Name: Labels_processed, Length: 201, dtype: object

In [13]:
df_capt_likes

,Labels,Likes,Labels_processed
0,"Forehead, Nose, Cheek, Chin, Eyebrow, Eyelash,...",7599,forehead nose cheek chin eyebrow eyelash jaw n...
1,"Hair, Forehead, Ear, Gesture, Black hair, Corn...",10139,hair forehead ear gesture black hair cornrows ...
2,"Clothing, Hair, Face, Skin, Head, Lip, Chin, O...",17172,clothing hair face skin head lip chin outerwea...
3,"Leg, Sleeve, Eyewear, Standing, Gesture, Stree...",26557,leg sleeve eyewear standing gesture street fas...
4,"Head, Outerwear, Shoe, Shoulder, Leg, Luggage ...",21524,head outerwear shoe shoulder leg luggage and b...
...,...,...,...
196,"Sky, Beige, Wood, Tints and shades, Facade, La...",44975,sky beige wood tints and shades facade landsca...
197,"Nose, Cheek, Skin, Lip, Lipstick, Eyelash, Mou...",27591,nose cheek skin lip lipstick eyelash mouth fla...
198,"Liquid, Product, Cosmetics, Material property,...",22501,liquid product cosmetics material property fon...
199,"Nose, Skin, Head, Lip, Lipstick, Chin, Shoulde...",39416,nose skin head lip lipstick chin shoulder eyeb...


In [14]:
dictionary = gensim.corpora.Dictionary(processed_labels)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break

0 cheek
1 chin
2 eyebrow
3 eyelash
4 forehead
5 neck
6 nose
7 sleev
8 black
9 caesar
10 child
11 cornrow
12 equip
13 gestur
14 hair
15 happi
16 person
17 protect
18 cloth
19 face
20 hairstyl


In [15]:
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_labels]
bow_corpus[10]

[(7, 1), (19, 1), (21, 2), (24, 1), (25, 1), (30, 1)]

In [16]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.453744461380033),
 (1, 0.41373596454864486),
 (2, 0.3133997175607922),
 (3, 0.3351590981671928),
 (4, 0.37372746771725673),
 (5, 0.3034363276229967),
 (6, 0.37372746771725673),
 (7, 0.20240915537711934)]


In [17]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.076*"photographi" + 0.070*"font" + 0.061*"automot" + 0.050*"tint" + 0.050*"shade" + 0.048*"gestur" + 0.046*"sleev" + 0.045*"flash" + 0.043*"rectangl" + 0.042*"wood"
Topic: 1 
Words: 0.085*"fashion" + 0.081*"sleev" + 0.049*"dress" + 0.048*"street" + 0.045*"waist" + 0.040*"photographi" + 0.040*"outerwear" + 0.038*"neck" + 0.038*"flash" + 0.035*"gestur"
Topic: 2 
Words: 0.097*"eyebrow" + 0.068*"eyelash" + 0.065*"nose" + 0.064*"forehead" + 0.054*"chin" + 0.053*"hair" + 0.039*"cheek" + 0.039*"mouth" + 0.038*"skin" + 0.035*"neck"
Topic: 3 
Words: 0.068*"blue" + 0.065*"electr" + 0.055*"human" + 0.053*"flash" + 0.053*"bodi" + 0.052*"photographi" + 0.039*"hair" + 0.034*"fashion" + 0.033*"eyelash" + 0.033*"light"
Topic: 4 
Words: 0.079*"white" + 0.072*"photographi" + 0.066*"black" + 0.065*"natur" + 0.063*"flash" + 0.040*"peopl" + 0.038*"human" + 0.037*"happi" + 0.032*"knee" + 0.032*"thigh"


**With the above 5 topics it becomes difficult to name the topics as there is a lot of overlap between some topics
. Reducing the topics to 4**

In [18]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))


Topic: 0 
Words: 0.098*"eyebrow" + 0.085*"eyelash" + 0.076*"nose" + 0.072*"forehead" + 0.061*"hair" + 0.057*"chin" + 0.052*"skin" + 0.049*"cheek" + 0.046*"head" + 0.045*"mouth"
Topic: 1 
Words: 0.070*"photographi" + 0.060*"flash" + 0.052*"white" + 0.049*"black" + 0.048*"sleev" + 0.040*"fashion" + 0.039*"waist" + 0.039*"shoulder" + 0.035*"human" + 0.034*"gestur"
Topic: 2 
Words: 0.124*"fashion" + 0.078*"sleev" + 0.068*"design" + 0.059*"street" + 0.041*"waist" + 0.034*"photographi" + 0.033*"wood" + 0.033*"flash" + 0.028*"outerwear" + 0.027*"automot"
Topic: 3 
Words: 0.070*"natur" + 0.060*"font" + 0.058*"photographi" + 0.048*"flash" + 0.043*"happi" + 0.042*"peopl" + 0.035*"rectangl" + 0.034*"sleev" + 0.033*"electr" + 0.032*"sunglass"


**Topic 0 Name: Headshot**

Words: 0.098*"eyebrow" + 0.085*"eyelash" + 0.076*"nose" + 0.072*"forehead" + 0.061*"hair" + 0.057*"chin" + 0.052*"skin" + 0.049*"cheek" + 0.046*"head" + 0.045*"mouth"

**Topic 1 Name: Black and White**

Words: 0.070*"photographi" + 0.060*"flash" + 0.052*"white" + 0.049*"black" + 0.048*"sleev" + 0.040*"fashion" + 0.039*"waist" + 0.039*"shoulder" + 0.035*"human" + 0.034*"gestur"

**Topic 2 Name: Street Background**

Words: 0.124*"fashion" + 0.078*"sleev" + 0.068*"design" + 0.059*"street" + 0.041*"waist" + 0.034*"photographi" + 0.033*"wood" + 0.033*"flash" + 0.028*"outerwear" + 0.027*"automot"

**Topic 3 Name: Nature background**
Words: 0.070*"natur" + 0.060*"font" + 0.058*"photographi" + 0.048*"flash" + 0.043*"happi" + 0.042*"peopl" + 0.035*"rectangl" + 0.034*"sleev" + 0.033*"electr" + 0.032*"sunglass"


In [19]:
count=0
for index, score in sorted(lda_model[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    count+=1
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9223963618278503	 
Topic: 0.098*"eyebrow" + 0.085*"eyelash" + 0.076*"nose" + 0.072*"forehead" + 0.061*"hair" + 0.057*"chin" + 0.052*"skin" + 0.049*"cheek" + 0.046*"head" + 0.045*"mouth"

Score: 0.02612437680363655	 
Topic: 0.070*"photographi" + 0.060*"flash" + 0.052*"white" + 0.049*"black" + 0.048*"sleev" + 0.040*"fashion" + 0.039*"waist" + 0.039*"shoulder" + 0.035*"human" + 0.034*"gestur"

Score: 0.025974715128540993	 
Topic: 0.124*"fashion" + 0.078*"sleev" + 0.068*"design" + 0.059*"street" + 0.041*"waist" + 0.034*"photographi" + 0.033*"wood" + 0.033*"flash" + 0.028*"outerwear" + 0.027*"automot"

Score: 0.025504523888230324	 
Topic: 0.070*"natur" + 0.060*"font" + 0.058*"photographi" + 0.048*"flash" + 0.043*"happi" + 0.042*"peopl" + 0.035*"rectangl" + 0.034*"sleev" + 0.033*"electr" + 0.032*"sunglass"


In [20]:
topic_1 = 'headshot'
topic_2 = 'black_and_white'
topic_3 = 'street_bg'
topic_4 = 'nature_bg'
topics_df = pd.DataFrame(columns=[topic_1,topic_2,topic_3, topic_4])
i = 0
for bow_corpus_i in bow_corpus:  
    for index, score in sorted(lda_model[bow_corpus_i], key=lambda tup: -1*tup[1]):
        #print()
        if index ==0:
            topics_df.loc[i, topic_1] = score
        if index ==1:
            topics_df.loc[i, topic_2] = score
        if index ==2:
            topics_df.loc[i, topic_3] = score
        if index ==3:
            topics_df.loc[i, topic_4] = score
           
    i = i+1
#posts_with_image_labels[fashion]
df_combined = pd.concat([df_capt_likes, topics_df], axis=1)
df_combined

,Labels,Likes,Labels_processed,headshot,black_and_white,street_bg,nature_bg
0,"Forehead, Nose, Cheek, Chin, Eyebrow, Eyelash,...",7599,forehead nose cheek chin eyebrow eyelash jaw n...,0.79143,0.0295182,0.150255,0.0287969
1,"Hair, Forehead, Ear, Gesture, Black hair, Corn...",10139,hair forehead ear gesture black hair cornrows ...,0.361591,0.563554,0.0370412,0.0378143
2,"Clothing, Hair, Face, Skin, Head, Lip, Chin, O...",17172,clothing hair face skin head lip chin outerwea...,0.922386,0.0261344,0.0259752,0.0255046
3,"Leg, Sleeve, Eyewear, Standing, Gesture, Stree...",26557,leg sleeve eyewear standing gesture street fas...,0.0250157,0.0265572,0.922875,0.0255523
4,"Head, Outerwear, Shoe, Shoulder, Leg, Luggage ...",21524,head outerwear shoe shoulder leg luggage and b...,0.0299132,0.913116,0.0287194,0.0282513
...,...,...,...,...,...,...,...
196,"Sky, Beige, Wood, Tints and shades, Facade, La...",44975,sky beige wood tints and shades facade landsca...,0.0632839,0.0626755,0.806738,0.0673024
197,"Nose, Cheek, Skin, Lip, Lipstick, Eyelash, Mou...",27591,nose cheek skin lip lipstick eyelash mouth fla...,0.901939,0.0332069,0.0321373,0.0327172
198,"Liquid, Product, Cosmetics, Material property,...",22501,liquid product cosmetics material property fon...,0.240107,0.0629267,0.0629751,0.633991
199,"Nose, Skin, Head, Lip, Lipstick, Chin, Shoulde...",39416,nose skin head lip lipstick chin shoulder eyeb...,0.905233,0.0319775,0.0313114,0.0314781


In [21]:
df_combined = df_combined.sort_values(by=['Likes'], ascending=False)
df_combined

,Labels,Likes,Labels_processed,headshot,black_and_white,street_bg,nature_bg
89,"Footwear, Face, Joint, Hand, Shoe, Shoulder, A...",232213,footwear face joint hand shoe shoulder arm dre...,0.0320056,0.904803,0.0315064,0.0316849
150,"Water, Sky, People on beach, Leg, People in na...",220363,water sky people on beach leg people in nature...,0.0315435,0.0339598,0.0330575,0.901439
88,"Outerwear, Shoulder, White, Dress shirt, Stree...",185766,outerwear shoulder white dress shirt street fa...,0.0211122,0.685063,0.272108,0.021717
56,"Outerwear, Fur clothing, Sleeve, Gesture, Head...",174832,outerwear fur clothing sleeve gesture headgear...,0.0315627,0.032409,0.904069,0.0319589
161,"Hair, Arm, Leg, Muscle, Human body, Flash phot...",167718,hair arm leg muscle human body flash photograp...,0.0327336,0.902441,0.0327636,0.0320618
...,...,...,...,...,...,...,...
22,"Window, Comfort, Cloud, Textile, Sleeve, Curta...",12303,window comfort cloud textile sleeve curtain wo...,0.0425108,0.0430219,0.870439,0.0440288
17,"Sleeve, Gesture, Fixture, Tints and shades, Re...",11812,sleeve gesture fixture tints and shades rectan...,0.0358722,0.0370319,0.0382517,0.888844
1,"Hair, Forehead, Ear, Gesture, Black hair, Corn...",10139,hair forehead ear gesture black hair cornrows ...,0.361591,0.563554,0.0370412,0.0378143
0,"Forehead, Nose, Cheek, Chin, Eyebrow, Eyelash,...",7599,forehead nose cheek chin eyebrow eyelash jaw n...,0.79143,0.0295182,0.150255,0.0287969


In [22]:
df_quartiles = df_combined.quantile([0.00, 0.25, 0.50, 0.75, 1.00])
bottom_quartile = df_combined[((df_combined['Likes']>=df_quartiles['Likes'][0.00]) & (df_combined['Likes']<df_quartiles['Likes'][0.25]))]
top_quartile = df_combined[((df_combined['Likes']>=df_quartiles['Likes'][0.75]) & (df_combined['Likes']<=df_quartiles['Likes'][1.00]))]
top_quartile

,Labels,Likes,Labels_processed,headshot,black_and_white,street_bg,nature_bg
89,"Footwear, Face, Joint, Hand, Shoe, Shoulder, A...",232213,footwear face joint hand shoe shoulder arm dre...,0.0320056,0.904803,0.0315064,0.0316849
150,"Water, Sky, People on beach, Leg, People in na...",220363,water sky people on beach leg people in nature...,0.0315435,0.0339598,0.0330575,0.901439
88,"Outerwear, Shoulder, White, Dress shirt, Stree...",185766,outerwear shoulder white dress shirt street fa...,0.0211122,0.685063,0.272108,0.021717
56,"Outerwear, Fur clothing, Sleeve, Gesture, Head...",174832,outerwear fur clothing sleeve gesture headgear...,0.0315627,0.032409,0.904069,0.0319589
161,"Hair, Arm, Leg, Muscle, Human body, Flash phot...",167718,hair arm leg muscle human body flash photograp...,0.0327336,0.902441,0.0327636,0.0320618
158,"Water, Sky, Flash photography, People in natur...",158412,water sky flash photography people in nature w...,0.0361926,0.0374211,0.219733,0.706654
108,"Glasses, Joint, Lip, Smile, Vision care, Shoul...",126158,glasses joint lip smile vision care shoulder f...,0.0324588,0.342816,0.0321659,0.59256
64,"Suit trousers, Outerwear, Vision care, Sleeve,...",121042,suit trousers outerwear vision care sleeve fla...,0.0230254,0.0238401,0.705086,0.248049
182,"Hairstyle, Photograph, White, Window, Street f...",116497,hairstyle photograph white window street fashi...,0.0283983,0.743206,0.199796,0.0285994
184,"Footwear, Jeans, Wheel, White, Street fashion,...",98991,footwear jeans wheel white street fashion leg ...,0.0360751,0.591269,0.336183,0.0364729


In [23]:
bottom_quartile

,Labels,Likes,Labels_processed,headshot,black_and_white,street_bg,nature_bg
143,"Sky, Mountain, Water, Nature, People in nature...",30278,sky mountain water nature people in nature nat...,0.0279951,0.0285834,0.0281097,0.915312
127,"Smile, White, Product, Black, Neck, Sleeve, Ge...",30070,smile white product black neck sleeve gesture ...,0.0283595,0.912974,0.0289544,0.0297117
114,"Human body, Flash photography, Happy, Dress, P...",28871,human body flash photography happy dress peopl...,0.0160504,0.0171785,0.515388,0.451383
66,"Flash photography, Font, Entertainment, Sky, E...",28323,flash photography font entertainment sky event...,0.0316066,0.0321774,0.0319719,0.904244
185,"Hair, Forehead, Nose, Cheek, Head, Lip, Chin, ...",28301,hair forehead nose cheek head lip chin eyebrow...,0.905786,0.0314882,0.0314168,0.0313095
180,"Chair, Rectangle, Font, Art, Tints and shades,...",27893,chair rectangle font art tints and shades mage...,0.041901,0.042602,0.164949,0.750547
197,"Nose, Cheek, Skin, Lip, Lipstick, Eyelash, Mou...",27591,nose cheek skin lip lipstick eyelash mouth fla...,0.901939,0.0332069,0.0321373,0.0327172
69,"Outerwear, Leg, Comfort, Sleeve, Knee, Chair, ...",27554,outerwear leg comfort sleeve knee chair blazer...,0.0315885,0.0321086,0.9033,0.0330031
146,"Glasses, Vision care, Light, Azure, Purple, Ey...",27342,glasses vision care light azure purple eyewear...,0.0282423,0.0288449,0.0293847,0.913528
181,"Forehead, Nose, Cheek, Skin, Lip, Chin, Hairst...",27297,forehead nose cheek skin lip chin hairstyle ey...,0.924682,0.0251797,0.0250799,0.0250579


In [24]:
print(top_quartile.mean())

Likes              89455.450980
headshot               0.122275
black_and_white        0.431159
street_bg              0.248762
nature_bg              0.197804
dtype: float64


In [25]:
print(bottom_quartile.mean())

Likes              20570.360000
headshot               0.177651
black_and_white        0.204861
street_bg              0.267869
nature_bg              0.349620
dtype: float64


**Task F**: What advice would you give Zara if it wants to increase engagement on its Instagram 
page based on your findings?

It is evident from the average weights of top quartiles that **black and white photos receive more likes** from the users and  **headshots receive the least**

From the bottom quartile average weights, it can be seen that headshots receive the least likes whereas nature background received the most likes

An advice to Zara would be to **post less headshot images and post more black and white flash photography images**. Black and white images seem to increase user engagement. If Zara wants to post headshot images, it should try posting black and white headshot images to increase user engagement.  

For the choice of background, **Zara should prefer street background over nature background** to increase engagement